In [2]:
import json
import numpy as np
import pandas as pd
import requests

In [3]:
key = open('APIkeys.txt', 'r')

In [4]:
keys = list(key)

In [5]:
api_key = keys[0].split()[2]

In [37]:
url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=Pete Wells&fq=news_desk:(Dining) AND type_of_material:(Review)&page={page}&api-key={api_key}'



In [38]:
r = requests.get(url)

In [39]:
json_data = r.json()

In [40]:
r.json()['response']['docs'][6]['web_url']

'https://www.nytimes.com/2018/09/18/dining/shoji-at-69-leonard-street-review.html'

In [44]:
def query_nytimes(api_key, n_page, news_desk = 'Dining', news_type = 'Review', query = 'Pete Wells'):
    url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&fq=news_desk:{news_desk} AND type_of_material:{news_type}&page={n_page}&api-key={api_key}'
    r = requests.get(url)
    json_data = r.json()
    return r.json()['response']['docs']

In [45]:
query_nytimes(api_key, 1)

 {},
    'subType': 'blogSmallInline',
    'crop_name': 'blogSmallInline'},
   {'rank': 0,
    'subtype': 'mediumFlexible177',
    'caption': None,
    'credit': None,
    'type': 'image',
    'url': 'images/2018/09/12/dining/12REST-slide-Q083/12REST-slide-Q083-mediumFlexible177.jpg',
    'height': 118,
    'width': 177,
    'legacy': {},
    'subType': 'mediumFlexible177',
    'crop_name': 'mediumFlexible177'},
   {'rank': 0,
    'subtype': 'videoSmall',
    'caption': None,
    'credit': None,
    'type': 'image',
    'url': 'images/2018/09/12/dining/12REST-slide-Q083/12REST-slide-Q083-videoSmall.jpg',
    'height': 281,
    'width': 500,
    'legacy': {},
    'subType': 'videoSmall',
    'crop_name': 'videoSmall'},
   {'rank': 0,
    'subtype': 'videoHpMedium',
    'caption': None,
    'credit': None,
    'type': 'image',
    'url': 'images/2018/09/12/dining/12REST-slide-Q083/12REST-slide-Q083-videoHpMedium.jpg',
    'height': 211,
    'width': 375,
    'legacy': {},
    'subType': 